In [1]:
import os
import sys
sys.path.insert(0, "..")

from tqdm import tqdm

import torch

from my_datasets import *

In [2]:
torch.manual_seed(101)
n, r, ap, bp, c = 16, 64, 0.2, 0.2, 3
for _ in tqdm(range(1000)):
    rdict = random_rules_with_chain(n, r, ap, bp, c, return_dict=True)
    rules, states = rdict["rules"], rdict["states"]
    qed = prove_theorem(rules[None,...], torch.ones(1,n), init_state = states[0:1])
    diff = (qed["states"].squeeze() - states).abs().sum()
    if diff > 0 or qed["chain_len"] != c:
        break

100%|██████████| 1000/1000 [00:01<00:00, 987.08it/s]


In [3]:
dataset = AutoregKStepsTokensDataset(
    num_vars = n,
    num_rules_range = (r, 2*r),
    ante_prob_range = (0.2, 0.3),
    conseq_prob_range = (0.2, 0.3),
    chain_len_range = (c, 2*c),
    num_prevs_range = (1, c),
    num_steps = 3,
    dataset_len = 1000,
)

In [4]:
for i in tqdm(range(1000)):
    item = dataset[i]
    tokens, labels = item["tokens"], item["labels"]
    qed = prove_theorem(tokens[None,...], torch.ones(1,n), tokens[-1][n:][None,...])
    states = qed["states"].squeeze()
    diff = (labels - states[1:dataset.num_steps+1]).abs().sum()
    if diff > 0:
        break

100%|██████████| 1000/1000 [00:01<00:00, 802.70it/s]


In [5]:
states

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 1., 1.],
        [1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1.],
        [1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 1., 1., 1., 1.],
        [1., 1., 0., 1., 0., 0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1.],
        [1., 1., 0., 1., 1., 0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1.],
        [1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 0., 1., 1., 0., 1., 1

In [6]:
labels

tensor([[1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1],
        [1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1],
        [1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1]])